# Automating Crypto Website API Pull

In [ ]:
#Importing libraries and extracting data

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'16',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '237779a3-04ce-4111-8588-568f3a625505',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
#Importing pandas and displaying complete data frame

import pandas as pd


pd.set_option('display.max_columns', None)
pd.set_option('dislay_max_rows', None)

In [ ]:
#Making the extracted data readable

df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
#Creating function to pull data from site and load it into a new csv file, or add new data to csv if exists
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'16',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '237779a3-04ce-4111-8588-568f3a625505',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime('now')
    df
    
    if not os.path.isfile(r'C:\Users\KaeTech\Python Basics\API.csv')
        df.to_csv(r'C:\Users\KaeTech\Python Basics\API.csv', header = 'column_names')
    else:
        df.to_csv(r'C:\Users\KaeTech\Python Basics\API.csv', mode = 'a', header = False)

In [ ]:
#Importing libraries for and automation to run the previous function every minute

import os
from time import time 
from time import sleep

for i in range(333):
    api_runner()
    print("API Runner Completed")
    sleep(60)
exit()

In [ ]:
#Testing if csv was created and works

df3 = pd.read_csv(r'C:\Users\KaeTech\Python Basics\API.csv')
df3

In [ ]:
#Setting numerical data to 3 decimal places 

pd.set_option('display.float_format', lambda x: '%.3f' % x)
df

In [ ]:
#Creating new data frame to check mean of data chasges over the past few minutes

df4 = df.groupby('name', sort=False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()

In [ ]:
#Creating data frame for better visual representation, resetting the index, and renaming colums
df5 = df4.stack().to_frame(name='percentages').reset_index().rename(columns={'level_1': 'percent_change', 'percentages': 'values'})
df5

In [ ]:
#Renaminf row names in Percent Change Column 

df5['percent_change'] = df5['percent_change'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])

In [ ]:
#Plotting the data frame 

sns.catplot(x='percent_change', y='values', hue='name', data =df5, kind = 'point')